In [1]:
#!pip install -q openai

In [2]:
import os

import openai
import tiktoken

In [3]:
model = "gpt-4"   # 8192 токена
max_tokens =  4000
temperature = 0.3
token_limit = 8000

In [4]:
# функция подсчета числа токенов
def num_tokens_from_messages(messages, model=model):
    encoding= tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # каждое сообщение следует за <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # если есть имя, то роль опускается
                num_tokens += -1  # роль всегда обязательна и всегда 1 токен
    num_tokens += 2  # в каждом ответе используется <im_start> помощник
    return num_tokens

In [5]:
# настройка роли асистента
SYSTEM = """  Вы полезный и умный помощник магазина Ашан.
Вы умеете по вопросу клиента очень точно определять его тему.
Проичитав сообщение от клиента вы точно можете сказать к какой теме данное сообщение относится.
Вы може определить основной интент запроса.
"""

ASSISTANT = """
 Определите тему сообщения:
- Если тема сообщения касается возврата некачественного продовольственного товара, то сообщите: "возврат продовольственного".
- Если тема сообщения касается возврата некачественного неподовольственного товара, то сообщите: "возврат непродовольственного".
- Если тема относится к товарам-исключениям, предметы личной гигиены и парфюмерно-косметические товары; швейные и трикотажные изделия; бытовая химия, пестициды и агрохимикаты; растения, то сообщите: "возврат исключения".
- Если тема об условиях доставки товара, то сообщите: "условия доставки".
- Если тема сообщения о стоимости бесплатной доставки, то сообщите "бесплатная доставка".
- Если тема сообщения не соотвествует выше перечисленным, сообщите: "другая тема".
"""

messages=[
    {"role": "system", "content": SYSTEM},
    {"role": "assistant", "content": ASSISTANT}
]

In [6]:
content = ''
while not 'пока' in content.lower():
    content = input("Ввод: ").strip()
    messages.append({"role": "user", "content": content})
    
    conv_history_tokens = num_tokens_from_messages(messages)
    print(f"Токенов: {conv_history_tokens}")
    while conv_history_tokens + max_tokens >= token_limit:
        del messages[2]
        conv_history_tokens = num_tokens_from_messages(messages)

    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens = max_tokens,
        temperature = temperature
    )
    chat_response = completion.choices[0].message.content

    print(f'''
GPT: {chat_response}
''')

    messages.append({"role": "assistant", "content": chat_response})

Ввод: добрый день!
Токенов: 347

GPT: Тема сообщения: "Приветствие"

Ввод: хочу вернуть товар
Токенов: 377

GPT: Тема сообщения: "Возврат товара"

Ввод: я купил неисправный миксер, что делать?
Токенов: 417

GPT: Тема сообщения: "возврат непродовольственного"

Ввод: вчера доставили чипсы с порванной упаковкой
Токенов: 468

GPT: Тема сообщения: "возврат продовольственного"

Ввод: купила помаду, упаковка была вскрыта
Токенов: 513

GPT: Тема сообщения: "возврат исключения"

Ввод: пока
Токенов: 540

GPT: Тема сообщения: "Прощание"



In [7]:
messages

[{'role': 'system',
  'content': '  Вы полезный и умный помощник магазина Ашан.\nВы умеете по вопросу клиента очень точно определять его тему.\nПроичитав сообщение от клиента вы точно можете сказать к какой теме данное сообщение относится.\nВы може определить основной интент запроса.\n'},
 {'role': 'assistant',
  'content': '\n Определите тему сообщения:\n- Если тема сообщения касается возврата некачественного продовольственного товара, то сообщите: "возврат продовольственного".\n- Если тема сообщения касается возврата некачественного неподовольственного товара, то сообщите: "возврат непродовольственного".\n- Если тема относится к товарам-исключениям, предметы личной гигиены и парфюмерно-косметические товары; швейные и трикотажные изделия; бытовая химия, пестициды и агрохимикаты; растения, то сообщите: "возврат исключения".\n- Если тема сообщения другая, сообщите: "другая тема".\n'},
 {'role': 'user', 'content': 'добрый день!'},
 {'role': 'assistant', 'content': 'Тема сообщения: "Приве